In [127]:
import fitz
from nltk.tokenize import sent_tokenize
import pandas as pd
import requests
import pdfkit

wkhtmltopdf_path = r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe"

## Download PDFs and store in `/data/raw/`

- use the links from `Exhibit-Excel-08-10-2025.xlsx` 

In [106]:
lexamples_export_path = r"..\data\Exhibit-Excel-08-10-2025.xlsx"

report_links = pd.read_excel(lexamples_export_path)
report_links.head()

,ID,CIK,Company,Sec Url,Lexample Url,Number,Filing Date,Description
0,5522470,1997377,IsoEnergy Ltd.,https://www.sec.gov/Archives/edgar/data/199737...,https://lexamples.com/exhibits/contents/NTUyMj...,99.1,2025-07-15,"News Release dated July 15, 2025 - IsoEnergy R..."
1,5520781,1513845,Yandex N.V.,https://www.sec.gov/Archives/edgar/data/151384...,https://lexamples.com/exhibits/contents/NTUyMD...,99.1,2025-07-10,Press release of Nebius Group N.V. dated July ...
2,5509529,1173420,NOVAGOLD RESOURCES INC,https://www.sec.gov/Archives/edgar/data/117342...,https://lexamples.com/exhibits/contents/NTUwOT...,99.1,2025-06-16,"Press release, dated June 16, 2025 issued by N..."
3,5500739,771992,PAN AMERICAN SILVER CORP,https://www.sec.gov/Archives/edgar/data/771992...,https://lexamples.com/exhibits/contents/NTUwMD...,99.1,2025-05-29,News Release - Pan American Silver Releases 20...
4,5497256,1402388,"WEST FRASER TIMBER CO., LTD",https://www.sec.gov/Archives/edgar/data/140238...,https://lexamples.com/exhibits/contents/NTQ5Nz...,99.1,2025-05-22,"News release dated May 22, 2025 re: West Frase..."


In [ ]:
# downloading reports, renaming and saving as /data/raw/<CompanyName_Year>.pdf

config = pdfkit.configuration(wkhtmltopdf=wkhtmltopdf_path)


# pdfkit.from_url(url=url, output_path='sample.pdf', configuration=config)

# maybe try a simple for loop lmao
# add a sleep before downloading next pdf

def download_pdfs(row):
    company = row['Company']
    url: str, filing_date: str
    pdfkit.from_url(url=url, output_path=fr'..\data\raw\{company}_{filing_date.split('-')[0]}.pdf', configuration=config)


report_links.loc[:5, ['Company', 'Lexample Url', 'Filing Date']].apply(download_pdfs()
                                                                      axis=1)

OSError: wkhtmltopdf reported an error:
Exit with code 1 due to network error: ProtocolUnknownError


In [ ]:
FOR 



True

## Preprocessing

- Extract text & segment into sentences

- Manually label ~500 sentences into ESG categories (small gold standard)

In [ ]:
def extract_all_text(pdf_path, skip_pages=0):
    """Extracts all text from a PDF document."""
    doc = fitz.open(pdf_path)
    full_text = ""
    for page_num in range(skip_pages, doc.page_count):
        page = doc.load_page(page_num)
        full_text += page.get_text("text") 
    doc.close()
    return full_text

In [ ]:
pdf_path = r"..\data\Exhibit-PDF-08-10-2025.pdf"

full_text = extract_all_text(pdf_path, skip_pages=5)

In [21]:
full_text[0:10000]

'99.1 - IsoEnergy Ltd.\nEX-99.1 2 ex991.htm NEWS RELEASE DATED JULY 15, 2025\nExhibit 99.1\n \n \n \n \nIsoEnergy Releases Inaugural Sustainability Report, Underscoring Commitment to Responsible Growth\nTORONTO, July 15, 2025 /CNW/ - IsoEnergy Ltd. ("IsoEnergy" or the "Company") (NYSE American: ISOU) (TSX: ISO) is pleased to announce the release of its inaugural Sustainability\nReport (the "Report") for the year ended December 31, 2024. The Report highlights IsoEnergy\'s progress in advancing its global uranium portfolio, with a focus on environmental stewardship,\nIndigenous partnerships, and responsible governance.\nThe Report marks a milestone in the Company\'s evolution, following its strategic acquisition of Consolidated Uranium Inc. in December 2023, establishing a diversified asset base across tier-one\nuranium jurisdictions. As IsoEnergy advances toward potential near-term production, sustainability remains central to its growth strategy. The full Report is available at\nhttps:

In [37]:
# try:
#     nltk.data.find('tokenizers/punkt')
#     nltk.data.find('tokenizers/punkt_tab')
# except LookupError:
#     nltk.download('punkt')
#     nltk.download('punkt_tab')

# when using a venv, use:
# python -m nltk.downloader <tokenizer>

In [32]:
sentences = sent_tokenize(full_text)

In [42]:
df = pd.DataFrame(sentences, columns=['sentences'])

df.head()

,sentences
0,99.1 - IsoEnergy Ltd.\nEX-99.1 2 ex991.htm NEW...
1,The Report highlights IsoEnergy's progress in ...
2,The Report marks a milestone in the Company's ...
3,As IsoEnergy advances toward potential near-te...
4,The full Report is available at\nhttps://www.i...


There are escape characters like \n in sentences, so we need to remove them.

In [55]:
df.loc[:, 'sentences'] = df['sentences'].replace('\n', ' ', regex=True)
df.head()

,sentences
0,99.1 - IsoEnergy Ltd. EX-99.1 2 ex991.htm NEWS...
1,The Report highlights IsoEnergy's progress in ...
2,The Report marks a milestone in the Company's ...
3,As IsoEnergy advances toward potential near-te...
4,The full Report is available at https://www.is...


In [100]:
df.to_excel(r"..\data\processed_sentences.xlsx", index=False, engine='xlsxwriter')

c:\Users\krish\Work\projects\esg-disclosure-classifier\venv\Lib\site-packages\xlsxwriter\worksheet.py:1249: UserWarning: Ignoring URL 'https://www.mckinsey.com/tr/our-insights/stakeholder-capitalism-report Customers Employees Investors and Shareholders Merchants Public Institutions Banks Cooperatives Media Universities Non-Governmental Organizations Suppliers Retail Suppliers GRI 2-6, GRI 2-25, GRI 2-26, GRI 2-29 707 30 Türkiye’s Hepsiburada Environmentally Responsible Operations Through Innovative Solutions Start Here Equal and Inclusive Corporate Culture Starts Here Social Good Starts Here Ethical and Transparent Governance Starts Here Annexes 2022 Sustainability Report Our Sustainability Approach KEY STAKEHOLDER GROUP COMMUNICATION CHANNEL COMMUNICATION FREQUENCY Employees Slack, Mail, Phone, Physical/Online meetings, HepsiOne meetings, Motivation meetings, HepsiFun activities, Employee Loyalty survey Bence system Continuous Investors and Shareholders Press Releases, Financial Repor